In [1]:
import pandas as pd

### Read in the data

In [2]:
uber = pd.read_csv("datasets/uber.csv")
uber.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [4]:
uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


### Clean the data

In [5]:
uber=uber.drop("Unnamed: 0", axis=1)
uber.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [6]:
uber['dropoff_longitude'].dropna()
uber['dropoff_latitude'].dropna()
uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   key                200000 non-null  object 
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 12.2+ MB


In [34]:
'''
We need to clean our latitude and longitude values
 This is because there are legitimate bounds to lat and long
 the limits are as such : latitude[-90,90], latitude[-180,180]
However, there is a bug with the input of coords so we could
 only make it work with -90 to 90 for both
'''

uber_filtered = uber[uber['pickup_longitude'] < 90]
uber_filtered = uber_filtered[uber_filtered['pickup_longitude'] > -90]
print(f"{uber_filtered['pickup_longitude'].max()}, {uber_filtered['pickup_longitude'].min()}")

uber_filtered = uber_filtered[uber_filtered['pickup_latitude'] < 90]
uber_filtered = uber_filtered[uber_filtered['pickup_latitude'] > -90]
print(f"{uber_filtered['pickup_latitude'].max()}, {uber_filtered['pickup_latitude'].min()}")

uber_filtered = uber_filtered[uber_filtered['dropoff_longitude'] < 90]
uber_filtered = uber_filtered[uber_filtered['dropoff_longitude'] > -90]
print(f"{uber_filtered['dropoff_longitude'].max()}, {uber_filtered['dropoff_longitude'].min()}")

uber_filtered = uber_filtered[uber_filtered['dropoff_latitude'] < 90]
uber_filtered = uber_filtered[uber_filtered['dropoff_latitude'] > -90]
print(f"{uber_filtered['dropoff_latitude'].max()}, {uber_filtered['dropoff_latitude'].min()}")


40.808425, -89.93333299999999
48.01876, -74.01551500000001
40.831932, -75.458978633981
45.031598, -74.01575


In [60]:
import geopy.distance
def transform_coords():
    result_list = []

    for index, row in uber_filtered.iterrows():
        coords_1 = (row['pickup_longitude'], row['pickup_latitude'])
        coords_2 = (row['dropoff_longitude'], row['dropoff_latitude'])

        result_list.append(geopy.distance.geodesic(coords_2, coords_1).miles)

    uber_filtered['distance_in_miles'] = result_list

# distances = transform_coords(uber_filtered)

# uber_filtered['distance_in_miles'] = distances
uber_filtered.head(10)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance_in_miles
0,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,0.290272
1,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,0.423525
2,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,3.008397
3,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,0.786895
4,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,3.349106
5,2011-02-12 02:27:09.0000006,4.9,2011-02-12 02:27:09 UTC,-73.969019,40.755910,-73.969019,40.755910,1,0.000000
6,2014-10-12 07:04:00.0000002,24.5,2014-10-12 07:04:00 UTC,-73.961447,40.693965,-73.871195,40.774297,5,6.446615
7,2012-12-11 13:52:00.00000029,2.5,2012-12-11 13:52:00 UTC,0.000000,0.000000,0.000000,0.000000,1,0.000000
8,2012-02-17 09:32:00.00000043,9.7,2012-02-17 09:32:00 UTC,-73.975187,40.745767,-74.002720,40.743537,1,1.909893
9,2012-03-29 19:06:00.000000273,12.5,2012-03-29 19:06:00 UTC,-74.001065,40.741787,-73.963040,40.775012,1,2.712677


In [37]:
uber_filtered['distance_in_miles']

0         0.290272
1         0.423525
2         3.008397
3         0.786895
4         3.349106
            ...   
199995    0.040030
199996    1.523166
199997    8.896701
199998    1.122834
199999    0.935749
Name: distance_in_miles, Length: 199986, dtype: float64

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

uber_cat = uber[['key']]
ordinal_encoder = OrdinalEncoder()
uber_cat_encoded = ordinal_encoder.fit_transform(uber_cat)
uber_cat_encoded[:25]

### Beginning of the modeling process

In [55]:
from sklearn.linear_model import LinearRegression

uber_labels = uber_filtered['fare_amount'].copy()
uber_prepared = uber_filtered.drop(columns=['key','fare_amount', 'pickup_datetime'])
lin_reg = LinearRegression()

lin_reg.fit(uber_prepared, uber_labels)

LinearRegression()

In [56]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(uber_prepared, uber_labels)

DecisionTreeRegressor()

In [59]:
some_data = uber_filtered.iloc[:5]
some_labels = uber_labels.iloc[:5]
some_data_prepared = some_data.drop(columns=['key','fare_amount', 'pickup_datetime'])
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

Predictions: [11.28937441 11.28665465 11.28662384 11.42995421 11.58010299]
Labels: [7.5, 7.7, 12.9, 5.3, 16.0]


In [58]:
uber_filtered.iloc[:5]

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance_in_miles
0,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,0.290272
1,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,0.423525
2,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,3.008397
3,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,0.786895
4,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,3.349106


In [98]:
uber_filtered = uber_filtered.drop(columns=['key'])

### Splitting the data into training and testing sets

In [100]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(uber_filtered, test_size=0.2, random_state=42)

uber_df = train_set.drop("fare_amount", axis=1)
uber_labels = train_set['fare_amount'].copy()

In [105]:
uber_df

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance_in_miles
70767,-73.975785,40.759228,-74.012048,40.709707,1,2.687397
15045,-73.989005,40.758418,-73.870420,40.773570,2,8.228981
11426,-73.991654,40.733589,-73.985400,40.745320,4,0.488381
62727,-73.975572,40.790523,-73.981047,40.782260,1,0.411346
174658,-74.002747,40.739875,-73.953458,40.802095,5,3.619924
...,...,...,...,...,...,...
119889,-73.995103,40.744703,-73.999397,40.756405,1,0.372537
103703,-73.952260,40.777623,-73.966674,40.764214,2,1.032139
131942,-73.997574,40.741568,-74.007283,40.706988,1,0.943726
146879,-73.991377,40.749513,-73.980572,40.759482,1,0.773243


In [106]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(uber_df, uber_labels)

LinearRegression()

In [107]:
from sklearn.metrics import mean_squared_error
import numpy as np

uber_predictions = lin_reg.predict(uber_df)
lin_mse = mean_squared_error(uber_labels, uber_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

9.9732756288447

### Sklearn Metrics

In [108]:
from sklearn.metrics import r2_score

r2 = r2_score(uber_labels, uber_predictions)
r2

0.0008496990348527689

In [109]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)
rf_model.fit(uber_df, uber_labels)

RandomForestRegressor(max_depth=8, random_state=42)

In [113]:
predictions = rf_model.predict(uber_df)
rf_r2_score = r2_score(uber_labels, predictions)
rf_r2_score

0.7585453720744637

### Start of Cross Val

In [114]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(uber_df, uber_labels)


DecisionTreeRegressor()

In [116]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, uber_df, uber_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([6.72609047, 6.11205232, 6.13817092, 7.4399559 , 6.20904244,
       6.1504755 , 6.06255278, 5.78683972, 6.86532806, 6.43035968])

In [137]:
## Work in Progress
from sklearn.model_selection import StratifiedKFold

target = uber_labels

uber_list = uber_df.values.tolist()
uber_labels_list = uber_labels.values.tolist()
skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(uber_list, uber_labels_list)
print(skf)

StratifiedKFold(n_splits=2, random_state=None, shuffle=False)


### Grid Search

In [136]:
## Work in Progress
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
forest_reg = RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)

grid_search = GridSearchCV(forest_reg, param_grid, cv=2,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(uber_df, uber_labels)

KeyboardInterrupt: 